# Set completeness

## Import libraries

In [ ]:
import os
import h5py
import numpy
from openquake.mbt.oqt_project import OQtProject, OQtModel, OQtSource

## Settings

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
prj_dir = os.path.dirname(project_pickle_filename)
compl_hdf5_filename = os.path.join(prj_dir, oqtkp.compl_hdf5_filename)
compl_data_folder = os.path.abspath(os.path.join(prj_dir, model.compl_data_folder))

print('Active model ID             : %s' % (model_id))
print('Completeness file           : %s' % (compl_hdf5_filename))
print('Data completeness folder ID : %s' % (compl_data_folder))

## Set default completeness

In [ ]:
fhdf5 = h5py.File(compl_hdf5_filename, 'a')
print('completeness filename: {:s}'.format(compl_hdf5_filename))

# Update/create group
if model_id in fhdf5.keys():
    print('Group exists; selecting group: %s' % (model_id))
    grp = fhdf5[model_id]
else:
    print('Creating group: %s' % (model_id))
    grp = fhdf5.create_group(model_id)

# Read completeness 
comw = fhdf5[model_id]['whole_catalogue']

# Update the completeness for all the area sources 
for key in sorted(model.sources):
    if model.sources[key].source_type == 'AreaSource':
        
        fle = os.path.join(compl_data_folder, '{:s}.csv'.format(key))
        com = numpy.loadtxt(fle, delimiter=',')
        print(com)
        
        if key in grp:
            del fhdf5[model_id][key] 
            print('Updating source %s' % (key))
            grp[key] = com
        else:
            print('Creating source %s' % (key))
            dataset = grp.create_dataset(key, data=comw)
fhdf5.close()